In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import os
DATA = os.getenv('DEXNET_DATA')
print(DATA)                     #Confirm if path is correct

/home/jibran/dexnet_ws/gqcnn/data/training/dex-net_2.0/tensors


In [5]:
arrays = {}
datapoint = '_05590.npz'                #Enter datapoint here (anywhere between _00000 and _06728, there are a total of 6.7 million datapoints i.e, 6728 x 1000). Change this value to just '.npz' to train on entire dataset
for filename in os.listdir(DATA):
    if filename.endswith(datapoint):
        arrays[filename.replace(datapoint, '')] = np.load('/home/jibran/dexnet_ws/gqcnn/data/training/dex-net_2.0/tensors/'+filename)

features = {}
for array in arrays:
    f = arrays[array]
    feature = f['arr_0.npy']
    features[array] = feature
print(features.keys())

dict_keys(['table_mask', 'depth_ims_raw', 'image_labels', 'hand_poses', 'depth_ims_tf_table', 'robust_ferrari_canny', 'binary_ims_tf', 'binary_ims_raw', 'force_closure', 'object_labels', 'depth_ims_raw_table', 'pose_labels', 'depth_ims_tf'])


In [6]:
#Inputs to feed into CNN
aligned_imgs = features['depth_ims_tf_table']
# print(aligned_imgs.shape)
gripper_depths = features['hand_poses'][:, 2]          #aligned_imgs and gripper_depths are the "X" of our model (Refer to gqcnn/Data/README.md for more info)
# print(gripper_depths.shape)
grasp_metrics = features['robust_ferrari_canny']       #Y of our model (Grasp metric)

#Verify shapes 

print(grasp_metrics.shape)
print(gripper_depths.shape)
print(aligned_imgs.shape)

(1000,)
(1000,)
(1000, 32, 32, 1)


In [9]:
#@title Imports

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math
import random

# example of defining the discriminator model
from keras.models import Sequential
from keras.optimizers import Adam
from keras import layers
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Lambda
from tensorflow.nn import local_response_normalization
from keras.utils.vis_utils import plot_model

from keras.layers.merge import concatenate
from keras.models import Model
# from tensorflow.keras import layers
from keras.layers import Dense, Input

In [10]:
def getGraspQualityVariable():
    input = Input(shape=(32, 32, 1), name="img")
    x1 = Conv2D(filters=64, kernel_size=7, activation='relu')(input)
    x2 = Conv2D(filters=64, kernel_size=5, activation='relu')(x1)
    x3 = Lambda(local_response_normalization)(x2)
    x4 = MaxPooling2D(pool_size=(2, 2), strides=2)(x3)
    x5 = Conv2D(filters=64, kernel_size=3, activation='relu')(x4)
    x6 = Conv2D(filters=64, kernel_size=3, activation='relu')(x5)
    x7 = Lambda(local_response_normalization)(x6)
    x8 = Flatten()(x7)
    x9 = Dense(1024, activation='relu')(x8)
    
    # plot_model(model, to_file='GraspQualityModel_plot.png', show_shapes=True, show_layer_names=True)
    return x9, input
    
getGraspQualityVariable()

(<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'dense')>,
 <KerasTensor: shape=(None, 32, 32, 1) dtype=float32 (created by layer 'img')>)

In [11]:
def getPointcloudModel():
    input = Input(shape=(1), name="z")
    x = Dense(16, input_dim=1, activation='relu')(input)
    return x, input

In [15]:
from keras.layers.merge import concatenate
def getDexnet2Model():
    grasp_model, input_1 = getGraspQualityVariable()
    pc_model, input_2 = getPointcloudModel()

    out = Dense(1024, activation='relu')

    # x = layers.concatenate([grasp_model, pc_model])
    merge = concatenate([grasp_model, pc_model])
    out_1 = Dense(1024, activation='relu')(merge)
    out_2 = Dense(2, activation='softmax')(out_1)
    model = Model(inputs=[input_1, input_2], outputs=out_2)
    plot_model(model, to_file='Dexnet2_plot.png', show_shapes=True, show_layer_names=True)
    return model

first_model = getDexnet2Model()

In [16]:
first_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_12 (Conv2D)              (None, 26, 26, 64)   3200        img[0][0]                        
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 22, 22, 64)   102464      conv2d_12[0][0]                  
__________________________________________________________________________________________________
lambda_6 (Lambda)               (None, 22, 22, 64)   0           conv2d_13[0][0]                  
____________________________________________________________________________________________

In [17]:
first_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [21]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='weights/first_model.weights.best.hdf5', verbose = 1, save_best_only=True)
x_train = [aligned_imgs, gripper_depths]                                   #Check model.summary() and model architecture in paper
y_train = grasp_metrics                                                    #valued between [0, 1] grasp robustness for the given grasp
first_model.fit(x_train,
          y_train,
          batch_size=32,
          epochs=1, 
          validation_split=0.2,
          callbacks=[checkpointer])

25/25 [==============================] - 5s 160ms/step - loss: 0.1218 - accuracy: 0.6123 - val_loss: 0.0000e+00 - val_accuracy: 0.4650

Epoch 00001: val_loss improved from inf to 0.00000, saving model to weights/first_model.weights.best.hdf5


In [22]:
print(y_train)

[0.         0.         0.         0.         0.00113762 0.
 0.00576939 0.         0.         0.         0.         0.
 0.         0.0058793  0.         0.00123383 0.         0.00164662
 0.         0.         0.         0.         0.         0.00113762
 0.         0.00576939 0.         0.         0.         0.
 0.         0.         0.0058793  0.         0.00123383 0.
 0.00164662 0.         0.         0.         0.         0.
 0.00113762 0.         0.00576939 0.         0.         0.
 0.         0.         0.         0.0058793  0.         0.00123383
 0.         0.00164662 0.         0.         0.         0.
 0.         0.00113762 0.         0.00576939 0.         0.
 0.         0.         0.         0.         0.0058793  0.
 0.00123383 0.         0.00164662 0.         0.         0.
 0.         0.         0.00113762 0.         0.00576939 0.
 0.         0.         0.         0.         0.         0.0058793
 0.         0.00123383 0.         0.00164662 0.         0.
 0.         0.         0.